In [7]:
import torch_geometric
from torch_geometric.datasets import Planetoid

# Load the dataset

In [8]:
dataset = Planetoid(root="tutorial", name= "Cora")

Processing...
Done!


 Dataset properties

In [13]:
print(dataset)
print("number of graphs:\t\t", len(dataset))
print("Number of classes:\t\t", dataset.num_classes)
print("Number of node features:\t", dataset.num_node_features)
print("Number of edge features:\t", dataset.num_edge_features)

Cora()
number of graphs:		 1
Number of classes:		 7
Number of node features:	 1433
Number of edge features:	 0


Dataset shapes

In [15]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/Users/medinils/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [27]:
# edge info
print("edge_index:\t\t", dataset.data.edge_index.shape)
print(dataset.edge_index)
#for example, edge 1 is connecting node 0 and 633

# train data info
print("train_mask:\t\t", dataset.data.train_mask.shape)
print(dataset.train_mask)
#tensor with bolean info representing with nodes are in which set

# data y and x
print("\n")
print("x:\t\t", dataset.x.shape)
print(dataset.x) # x is the node feature. The number of nodes multiple for the number of node features
print("\n")
print("y:\t\t", dataset.y.shape) # y is the node labelling. It give us different categories
print(dataset.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [28]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [30]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

 Define our neural network


In [35]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max")
    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [37]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


In [38]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'

    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7160, Test: 0.7150
Epoch: 020, Val: 0.7180, Test: 0.7080
Epoch: 030, Val: 0.7180, Test: 0.7080
Epoch: 040, Val: 0.7180, Test: 0.7080
Epoch: 050, Val: 0.7180, Test: 0.7080
Epoch: 060, Val: 0.7260, Test: 0.7060
Epoch: 070, Val: 0.7260, Test: 0.7060
Epoch: 080, Val: 0.7280, Test: 0.7150
Epoch: 090, Val: 0.7280, Test: 0.7150
